In [2]:
import pandas as pd

In [3]:
tmap = pd.read_csv('../data/Tmap_2005.csv')

In [4]:
c1 = tmap['지역(시도)'] == '서울'
select_list = ['공공기관', '의료시설', '연구기관', '업종별기업', '행정기관', '종교', '가정의례']
# c2 = tmap['지역(시군구)'] == '강남구'
# c2 = tmap['일자']==20200510
seoul = tmap.loc[c1,:]
seoul.shape

(23250, 8)

In [5]:
l = []
for i in select_list:
    select_condition2 = tmap['검색지유형2']==i
    seoul_index = seoul.loc[select_condition2,:].index
    # l.append(seoul_index)
    l.extend(seoul_index)
new_seoul = seoul.drop(l)
new_seoul['검색지유형3'].unique()
# seoul['검색지유형2'].unique()
# seoul['검색지유형3'].unique()

array(['아파트브랜드', '지하철역', '아파트단지', '백화점', '아파트건설사', '공원', '할인점', '주상복합',
       '농수축산물시장', '대학교', '쇼핑센터', '테마파크', '주요빌딩', '일반상가', '산업시설기타',
       '고속도로 IC', '오피스텔', '스케이트장', '기차역', '전망대', '주차장', '박물관/기념관', '호텔',
       '전자제품양판점', '도서관', '극장', '커피전문점', '고등학교', '종합시장', '서적브랜드', '전문상가',
       '한식', '미술관', '드라이브코스', '문화유적지', '고궁', '버스터미널', '초등학교', '전시장',
       '수족관', '부페', '공항', '농수축산물점', '교차로', '학교내시설물', '유원지', '재래시장', '방송사',
       '운동장/체육관', '슈퍼마켓', '공연장', '섬', '분식', '기념비', '중학교', '아파트형공장',
       '종합스포츠센타', '골프장', '대한상공회의소', '은행', '주요건물기타', '주유소', '자동차영업소',
       '중고차매매', '대교/철교', '사회복지시설', '화랑/전시관', '삼성', '센터', '능묘', '회관',
       '도시고속화도로IC', '축제', '운전면허시험장', 'LG유플러스', '호수', '한샘가구', '관광안내소',
       '장난감', '동식물원', '패밀리레스토랑', '놀이시설'], dtype=object)

In [34]:
# 서울시 구
gu_list = new_seoul['지역(시군구)'].unique()
# 점수주기
new_seoul['point'] = 31-new_seoul['검색지랭킹(시군구내)']
# 지역별 점수 합산
result = pd.DataFrame()
for i in gu_list:
    con_gu = new_seoul['지역(시군구)'] == i
    # print(i)
    gu_seoul = new_seoul.loc[con_gu, :]
    s = gu_seoul.groupby(by=['검색지명']).sum()
    s = s.loc[s['point']>20,:]
    ss = s.sort_values(by='point', ascending=False).reset_index()
    ss['gu'] = i
    result=result.append(ss[['검색지명', 'point', 'gu']])
result.head(30)


,검색지명,point,gu
0,고덕래미안힐스테이트아파트,930,강동구
1,천호역,904,강동구
2,고덕그라시움아파트,868,강동구
3,강동롯데캐슬퍼스트아파트,795,강동구
4,현대백화점천호점,770,강동구
5,고덕아르테온아파트,727,강동구
6,고덕롯데캐슬베네루체아파트,697,강동구
7,암사선사현대아파트,650,강동구
8,삼익그린2차아파트,632,강동구
9,프라이어팰리스아파트,592,강동구


# 카카오 api
- 카카오 api 키워드로 장소검색을 사용하면 각각의 네비게이션 목적지들의 좌표값이 나온다.
- 지하철 역일경우에는 인구 분포가 안나오는데 그외에는 나온다.

In [35]:
name = result.iloc[20,0]

In [36]:
# cnt_df.plot(kind='bar')
import requests
from decouple import config
ka_key = config('KAKAO_RESTAPI')
kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='+name
h = {
    'Authorization': 'KakaoAK '+ka_key
}
res = requests.get(kakao_url, headers=h)
location_data = res.json()['documents']
# 구, 동 따로 구별해서 저장해주기
address = location_data[0]['address_name']
place_url = location_data[0]['place_url']
place_x = location_data[0]['x']
place_y = location_data[0]['y']
# print(place_x, place_y, address)
location_data

[{'address_name': '서울 강동구 길동 391-15',
  'category_group_code': 'SW8',
  'category_group_name': '지하철역',
  'category_name': '교통,수송 > 지하철,전철 > 수도권5호선',
  'distance': '',
  'id': '21160663',
  'phone': '02-6311-5491',
  'place_name': '길동역 5호선',
  'place_url': 'http://place.map.kakao.com/21160663',
  'road_address_name': '서울 강동구 양재대로 지하 1480',
  'x': '127.140021295948',
  'y': '37.5378264874159'},
 {'address_name': '서울 강동구 길동 391-4',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '가정,생활 > 미용 > 미용실 > 준오헤어',
  'distance': '',
  'id': '27387784',
  'phone': '02-472-0605',
  'place_name': '준오헤어 길동역점',
  'place_url': 'http://place.map.kakao.com/27387784',
  'road_address_name': '서울 강동구 양재대로 1484',
  'x': '127.140383871105',
  'y': '37.5380963568379'},
 {'address_name': '서울 강동구 길동 389-5',
  'category_group_code': 'BK9',
  'category_group_name': '은행',
  'category_name': '금융,보험 > 금융서비스 > 은행 > 우리은행',
  'distance': '',
  'id': '8473356',
  'phone': '02-478-9191',
  'plac

In [37]:
add_sp=address.split()
place_dong, place_gu = add_sp[2], add_sp[1]
print(place_gu, place_dong)

강동구 길동


In [38]:
kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CE7&x={place_x}&y={place_y}&radius=300'
cate_res = requests.get(kakao_category_url, headers=h)
cate_res.json()

{'documents': [{'address_name': '서울 강동구 길동 378-3',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
   'distance': '98',
   'id': '25017323',
   'phone': '1522-3232',
   'place_name': '스타벅스 길동역점',
   'place_url': 'http://place.map.kakao.com/25017323',
   'road_address_name': '서울 강동구 양재대로 1487',
   'x': '127.140059161042',
   'y': '37.5387130260926'},
  {'address_name': '서울 강동구 길동 367-14',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '음식점 > 카페',
   'distance': '130',
   'id': '1832772086',
   'phone': '010-6669-6609',
   'place_name': '빈터커피로스터스 길동점',
   'place_url': 'http://place.map.kakao.com/1832772086',
   'road_address_name': '서울 강동구 양재대로115길 10',
   'x': '127.139788117831',
   'y': '37.5389872510917'},
  {'address_name': '서울 강동구 길동 366-5',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '음식점 > 카페',
   'distance': '123',
   'id': '973086101',
  

In [32]:
len(cate_res.json()['documents'])

15